## Cross-Lingual Voice Clone Demo


In [1]:
import os
import torch
from openvoice import se_extractor
from openvoice.api import ToneColorConverter

c:\Users\Taniyow\miniconda3\envs\openvoice\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



### Initialization


In [2]:
torch.cuda.is_available()

True

In [3]:
ckpt_converter = "checkpoints/converter"
device = "cuda:0" if torch.cuda.is_available() else "cpu"
output_dir = "outputs"

tone_color_converter = ToneColorConverter(
    f"{ckpt_converter}/config.json", device=device
)
tone_color_converter.load_ckpt(f"{ckpt_converter}/checkpoint.pth")

os.makedirs(output_dir, exist_ok=True)

c:\Users\Taniyow\miniconda3\envs\openvoice\lib\site-packages\torch\nn\utils\weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
c:\Users\Taniyow\miniconda3\envs\openvoice\lib\site-packages\wavmark\__init__.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`.

Loaded checkpoint 'checkpoints/converter/checkpoint.pth'
missing/unexpected keys: [] []


In [4]:
print("Device used:", device)

Device used: cuda:0


In this demo, we will use OpenAI TTS as the base speaker to produce multi-lingual speech audio. The users can flexibly change the base speaker according to their own needs. Please create a file named `.env` and place OpenAI key as `OPENAI_API_KEY=xxx`. We have also provided a Chinese base speaker model (see `demo_part1.ipynb`).


In [5]:
from openai import OpenAI
from dotenv import load_dotenv

# Please create a file named .env and place your
# OpenAI key as OPENAI_API_KEY=xxx
load_dotenv()

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

response = client.audio.speech.create(
    model="tts-1",
    voice="nova",
    input="This audio will be used to extract the base speaker tone color embedding. "
    + "Typically a very short audio should be sufficient, but increasing the audio "
    + "length will also improve the output audio quality.",
)

response.stream_to_file(f"{output_dir}/openai_source_output.mp3")

C:\Users\Taniyow\AppData\Local\Temp\ipykernel_16224\1565073473.py:18: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(f"{output_dir}/openai_source_output.mp3")


### Obtain Tone Color Embedding


The `source_se` is the tone color embedding of the base speaker.
It is an average for multiple sentences with multiple emotions
of the base speaker. We directly provide the result here but
the readers feel free to extract `source_se` by themselves.


In [6]:
base_speaker = f"{output_dir}/openai_source_output.mp3"
source_se, audio_name = se_extractor.get_se(
    base_speaker, tone_color_converter, vad=True
)

# reference_speaker = (
#     "resources/example_reference.mp3"  # This is the voice you want to clone
# )
reference_speaker = "resources/My Apartment in Slow Austrian German _ Super Easy German 245 - Easy German (youtube).mp3"  # This is the voice you want to clone
target_se, audio_name = se_extractor.get_se(
    reference_speaker, tone_color_converter, vad=True
)

OpenVoice version: v1
[(0.0, 12.336)]
after vad: dur = 12.336


c:\Users\Taniyow\miniconda3\envs\openvoice\lib\site-packages\torch\functional.py:666: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\SpectralOps.cpp:878.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]


OpenVoice version: v1
[(3.278, 68.562), (70.702, 80.082), (80.75, 108.402), (109.71, 117.202), (117.806, 123.218), (126.062, 147.186), (148.974, 185.874), (186.286, 194.034), (195.182, 202.834), (203.502, 206.258), (208.75, 211.41), (211.726, 222.13), (222.254, 249.618), (253.934, 255.73), (256.046, 259.154), (259.95, 293.746), (296.27, 299.922), (301.486, 307.89), (309.582, 313.138), (317.742, 334.93), (345.166, 350.61), (358.286, 390.354), (391.438, 402.514), (406.286, 427.666), (430.894, 516.626), (517.646, 602.866), (603.406, 605.554), (606.958, 649.778), (654.51, 661.266), (661.902, 676.146), (684.11, 714.962), (716.206, 734.93), (738.286, 760.05), (761.006, 765.458), (767.214, 776.082), (776.174, 779.41), (779.438, 793.906), (797.454, 844.178)]
after vad: dur = 757.305


### Inference


In [9]:
# Run the base speaker tts
text = [
    "MyShell is a decentralized and comprehensive platform for discovering, creating, and staking AI-native apps.",
    "MyShell es una plataforma descentralizada y completa para descubrir, crear y apostar por aplicaciones nativas de IA.",
    "MyShell est une plateforme décentralisée et complète pour découvrir, créer et miser sur des applications natives d'IA.",
    "MyShell ist eine dezentralisierte und umfassende Plattform zum Entdecken, Erstellen und Staken von KI-nativen Apps.",
    "MyShell è una piattaforma decentralizzata e completa per scoprire, creare e scommettere su app native di intelligenza artificiale.",
    "MyShellは、AIネイティブアプリの発見、作成、およびステーキングのための分散型かつ包括的なプラットフォームです。",
    "MyShell — это децентрализованная и всеобъемлющая платформа для обнаружения, создания и стейкинга AI-ориентированных приложений.",
    "MyShell هي منصة لامركزية وشاملة لاكتشاف وإنشاء ورهان تطبيقات الذكاء الاصطناعي الأصلية.",
    "MyShell是一个去中心化且全面的平台，用于发现、创建和投资AI原生应用程序。",
    "MyShell एक विकेंद्रीकृत और व्यापक मंच है, जो AI-मूल ऐप्स की खोज, सृजन और स्टेकिंग के लिए है।",
    "MyShell é uma plataforma descentralizada e abrangente para descobrir, criar e apostar em aplicativos nativos de IA.",
]

# Define the language codes for each text entry
language_codes = [
    "english",  # en
    "spanish",  # es
    "french",  # fr
    "german",  # de
    "italian",  # it
    "japanese",  # ja
    "russian",  # ru
    "arabic",  # ar
    "chinese",  # zh
    "hindi",  # hi
    "portuguese",  # pt
]
# Ensure that the number of language codes matches the number of texts
assert len(language_codes) == len(
    text
), "Language codes and text entries count mismatch."

src_path = f"{output_dir}/tmp.wav"

for i, (t, lang_code) in enumerate(zip(text, language_codes)):

    response = client.audio.speech.create(
        model="tts-1",
        voice="nova",
        input=t,
    )

    response.stream_to_file(src_path)

    save_path = f"{output_dir}/output_{lang_code}.wav"

    # Run the tone color converter
    encode_message = "@MyShell"
    tone_color_converter.convert(
        audio_src_path=src_path,
        src_se=source_se,
        tgt_se=target_se,
        output_path=save_path,
        message=encode_message,
    )

C:\Users\Taniyow\AppData\Local\Temp\ipykernel_16224\1534524028.py:45: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(src_path)
